In [1]:
import locale
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
summary = pd.read_csv('data/atussum_2014.dat')

In [3]:
summary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11592 entries, 0 to 11591
Columns: 409 entries, tucaseid to t500107
dtypes: float64(1), int64(408)
memory usage: 36.3 MB


In [4]:
summary.keys()

Index(['tucaseid', 'TUFINLWGT', 'TRYHHCHILD', 'TEAGE', 'TESEX', 'PEEDUCA',
       'PTDTRACE', 'PEHSPNON', 'GTMETSTA', 'TELFS', 
       ...
       't181601', 't181699', 't181801', 't181899', 't189999', 't500101',
       't500103', 't500105', 't500106', 't500107'],
      dtype='object', length=409)

May want to rename columns

* TUFINLWGT = statistical weight of respondent
* TRYHHCHILD = age of youngest child in household
* TEAGE - age of child
* TESEX - sex of child
* TELFS - working status of respondent
* TRCHILDNUM - number of children in household

Lookup in atusintcodebk

In [5]:
summary.head()

,tucaseid,TUFINLWGT,TRYHHCHILD,TEAGE,TESEX,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,...,t181601,t181699,t181801,t181899,t189999,t500101,t500103,t500105,t500106,t500107
0,20140101140007,4891315.826760,0,20,2,39,1,1,1,4,...,0,0,0,0,0,0,0,0,0,0
1,20140101140011,23147368.359655,16,16,2,37,2,2,1,1,...,0,0,0,0,0,0,0,0,0,0
2,20140101140028,20941823.429044,2,65,2,38,1,1,1,5,...,0,0,0,0,0,0,0,0,0,0
3,20140101140063,2505899.417166,-1,50,2,39,2,2,2,5,...,0,0,0,0,0,0,0,0,0,0
4,20140101140168,15928231.823165,8,23,2,39,2,2,2,4,...,0,0,0,0,0,0,0,0,0,0


In [6]:
len(summary.tucaseid.unique())

11592

In [8]:
# Households 
summary[summary.TRCHILDNUM == 0].head()

,tucaseid,TUFINLWGT,TRYHHCHILD,TEAGE,TESEX,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,...,t181601,t181699,t181801,t181899,t189999,t500101,t500103,t500105,t500106,t500107
3,20140101140063,2505899.417166,-1,50,2,39,2,2,2,5,...,0,0,0,0,0,0,0,0,0,0
5,20140101140537,3622350.817628,-1,85,1,36,2,2,1,5,...,0,0,0,0,0,0,0,0,0,0
8,20140101140614,2019096.316437,-1,63,2,40,1,2,1,1,...,0,0,0,0,0,0,0,0,70,0
9,20140101140639,5332005.152751,-1,85,1,44,1,2,1,5,...,0,0,0,0,0,0,0,0,0,0
10,20140101140665,4778595.528829,-1,66,2,39,1,2,1,1,...,0,0,0,0,0,0,0,0,0,0


In [9]:
activity = pd.read_csv('data/atusact_2014.dat')

In [10]:
activity.head()

,TUCASEID,TUACTIVITY_N,TEWHERE,TRTCCTOT_LN,TRTCC_LN,TRTCOC_LN,TRTEC_LN,TRTHH_LN,TRTNOHH_LN,TRTOHH_LN,...,TUCUMDUR24,TUEC24,TUSTARTTIM,TUSTOPTIME,TUTIER1CODE,TUTIER2CODE,TUTIER3CODE,TRCODE,TRTIER2,TXWHERE
0,20140101140007,1,-1,0,0,0,-1,0,-1,0,...,540,-1,04:00:00,13:00:00,1,1,1,10101,101,0
1,20140101140007,2,1,0,0,0,-1,0,-1,0,...,570,-1,13:00:00,13:30:00,3,1,1,30101,301,0
2,20140101140007,3,-1,60,60,0,-1,60,-1,60,...,630,-1,13:30:00,14:30:00,1,2,1,10201,102,0
3,20140101140007,4,13,10,10,0,-1,10,-1,10,...,640,-1,14:30:00,14:40:00,18,11,1,181101,1811,0
4,20140101140007,5,4,30,30,0,-1,30,-1,30,...,670,-1,14:40:00,15:10:00,11,1,1,110101,1101,0


In [19]:
summary = summary.rename(columns={'tucaseid': 'TUCASEID'})
summary.head(2)

,TUCASEID,TUFINLWGT,TRYHHCHILD,TEAGE,TESEX,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,...,t181601,t181699,t181801,t181899,t189999,t500101,t500103,t500105,t500106,t500107
0,20140101140007,4891315.826760,0,20,2,39,1,1,1,4,...,0,0,0,0,0,0,0,0,0,0
1,20140101140011,23147368.359655,16,16,2,37,2,2,1,1,...,0,0,0,0,0,0,0,0,0,0


In [22]:
merged = pd.merge(summary, activity)

In [23]:
len(merged[merged.TRCHILDNUM == 0])

122348